In [2]:
from convert import csv_to_dataset
from sklearn.model_selection import train_test_split

therapy = csv_to_dataset("Sheet_1.csv","response_text","class")
X_train, X_test, y_train, y_test = train_test_split(therapy["data"], therapy["target"], test_size=0.33, random_state=42)

{'data': ['I try and avoid this sort of conflict', 'Had a friend open up to me about his mental addiction to weed and how it was taking over his life and making him depressed', 'I saved a girl from suicide once. She was going to swallow a bunch of pills and I talked her out of it in a very calm, loving way.', 'i cant think of one really...i think i may have indirectly', 'Only really one friend who doesn\'t fit into the any of the above categories. Her therapist calls it spiraling." Anyway she pretty much calls me any time she is frustrated by something with  her boyfriend to ask me if it\'s logical or not. Before they would just fight and he would call her crazy. Now she asks me if it\'s ok he didn\'t say "please" when he said  "hand me the remote."', 'a couple of years ago my friends was going to switch school because of low self esteem too. I helped him overcome that shit too', 'Roommate when he was going through death and loss of a gf. Did anything to get him out of his bedroom.', "

In [3]:
#efficient version
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect',CountVectorizer()),
                    ('tfidf',TfidfTransformer()),
                    ('clf',MultinomialNB())])



text_clf = text_clf.fit(twenty_train.data,twenty_train.target)

### Construct NLTK stemmer

In [4]:
import nltk
# nltk.download()
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english", ignore_stopwords=True)
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
class LemmatizedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(LemmatizedCountVectorizer, self).build_analyzer()
        return lambda doc: ([lemmatizer.lemmatize(w) for w in analyzer(doc)])
stemmed_count_vect = StemmedCountVectorizer(stop_words='english',ngram_range=(1,2))
lemmatized_count_vect = LemmatizedCountVectorizer(ngram_range=(1,2))

## Fully loaded Pipeline

In [22]:
text_clf_svm = Pipeline([('vect',lemmatized_count_vect), # could also be stemmed, lemmatized is marginally better
                    ('tfidf',TfidfTransformer(use_idf=True)),
                    ('clf_svm',SGDClassifier(loss="hinge",alpha=0.0001,random_state=42)),
                   # ('clf_svm',MultinomialNB()) #can also pass MultiNomialNB
                    ])
text_clf_svm = text_clf_svm.fit(X_train,y_train)

### PRedictions before grid search

In [23]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',shuffle=True)
predicted = text_clf_svm.predict(X_test)
np.mean(predicted == y_test)

0.7407407407407407

### Refine with GridSearch

In [24]:
from sklearn.model_selection import GridSearchCV
"""
best params: default params
{'clf_svm__alpha': 0.001, #or .0005
 'clf_svm__early_stopping': False,
 'clf_svm__penalty': 'l2',
 'vect__ngram_range': (1, 2)}
"""
params_svm= {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf':(True,False),
    'clf_svm__penalty':['l2'],
    'clf_svm__alpha':[0.001,.0001,.0005],
}
params_nb= {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf':(True,False),
    'clf_svm__alpha':[1,.7,.5],
}

gs_clf = GridSearchCV(text_clf_svm,params_svm,n_jobs=-1,verbose = 2)
gs_clf = gs_clf.fit(X_train,y_train)
gs_clf.best_params_

Fitting 5 folds for each of 12 candidates, totalling 60 fits


{'clf_svm__alpha': 0.001,
 'clf_svm__penalty': 'l2',
 'tfidf__use_idf': True,
 'vect__ngram_range': (1, 2)}

In [25]:
gs_clf.best_score_

0.8272727272727274

In [26]:
twenty_test = fetch_20newsgroups(subset='test',shuffle=True)
predicted = gs_clf.predict(X_test)
np.mean(predicted == y_test)

0.6666666666666666

In [10]:
import time
import random
count = 1
start = time.time()
for i in range(10**7):
    count+=random.random()
print(time.time()-start)

1.5169978141784668
